In [1]:
import sympy as sp

In [21]:
vx, vy, vz = sp.symbols('v.x() v.y() v.z()')
wx, wy, wz = sp.symbols('w.x() w.y() w.z()')
A, B, C = sp.symbols('A B C')
th = sp.symbols('θ')

In [22]:
def hat(x):
    return sp.Matrix([[0, -x[2], x[1]], [x[2], 0, -x[0]], [-x[1], x[0], 0]])

def diff_vec(q, symbols):
    return sp.Matrix.hstack(*[q.diff(s) for s in symbols])

def diff_mat(Q, symbols):
    return sp.Matrix.hstack(*[diff_vec(Q[i, :].transpose(), symbols) for i in range(Q.rows)])

def show(expr):
    print(expr)
    return expr

In [23]:
v = sp.Matrix([vx, vy, vz])
w = sp.Matrix([wx, wy, wz])
V = hat(v)
W = hat(w)

# SO3

In [24]:
A_so3 = (1 - sp.cos(th)) / th**2
B_so3 = (th - sp.sin(th)) / th**3
A_so3inv = 1 / th**2 - (1 + sp.cos(th)) / (2 * th * sp.sin(th))

In [25]:
A_so3.series(th, 0, 3)

1/2 - θ**2/24 + O(θ**3)

In [26]:
B_so3.series(th, 0, 3)

1/6 - θ**2/120 + O(θ**3)

In [27]:
A_so3inv.series(th, 0, 3)

1/12 + θ**2/720 + O(θ**3)

In [28]:
show((A_so3.diff()/th).expand())

sin(θ)/θ**3 + 2*cos(θ)/θ**4 - 2/θ**4


sin(θ)/θ**3 + 2*cos(θ)/θ**4 - 2/θ**4

In [29]:
show((B_so3.diff()/th).expand())

-cos(θ)/θ**4 - 2/θ**4 + 3*sin(θ)/θ**5


-cos(θ)/θ**4 - 2/θ**4 + 3*sin(θ)/θ**5

In [30]:
show((A_so3inv.diff()/th).expand())

1/(2*θ**2) + cos(θ)**2/(2*θ**2*sin(θ)**2) + cos(θ)/(2*θ**2*sin(θ)**2) + cos(θ)/(2*θ**3*sin(θ)) + 1/(2*θ**3*sin(θ)) - 2/θ**4


1/(2*θ**2) + cos(θ)**2/(2*θ**2*sin(θ)**2) + cos(θ)/(2*θ**2*sin(θ)**2) + cos(θ)/(2*θ**3*sin(θ)) + 1/(2*θ**3*sin(θ)) - 2/θ**4

In [51]:
show(diff_mat(-a * hat(w) + b * hat(w) * hat(w), [wx, wy, wz]))

Matrix([[0, -2*B*w.y(), -2*B*w.z(), B*w.y(), B*w.x(), -A, B*w.z(), A, B*w.x()], [B*w.y(), B*w.x(), A, -2*B*w.x(), 0, -2*B*w.z(), -A, B*w.z(), B*w.y()], [B*w.z(), -A, B*w.x(), A, B*w.z(), B*w.y(), -2*B*w.x(), -2*B*w.y(), 0]])


Matrix([
[      0, -2*B*w.y(), -2*B*w.z(),    B*w.y(), B*w.x(),         -A,    B*w.z(),          A, B*w.x()],
[B*w.y(),    B*w.x(),          A, -2*B*w.x(),       0, -2*B*w.z(),         -A,    B*w.z(), B*w.y()],
[B*w.z(),         -A,    B*w.x(),          A, B*w.z(),    B*w.y(), -2*B*w.x(), -2*B*w.y(),       0]])

# SE3

In [32]:
A_se3 = (th - sp.sin(th)) / th**3
B_se3 = (sp.cos(th) - 1 + th**2 / 2) / th**4
C_se3 = (th - sp.sin(th) - th**3 / 6) / th**5

In [33]:
A_se3.series(th, 0, 3)

1/6 - θ**2/120 + O(θ**3)

In [34]:
B_se3.series(th, 0, 3)

1/24 - θ**2/720 + O(θ**3)

In [35]:
C_se3.series(th, 0, 3)

-1/120 + θ**2/5040 + O(θ**3)

In [45]:
show((A_se3.diff()/th).expand())

-cos(θ)/θ**4 - 2/θ**4 + 3*sin(θ)/θ**5


-cos(θ)/θ**4 - 2/θ**4 + 3*sin(θ)/θ**5

In [46]:
show((B_se3.diff()/th).expand())

-1/θ**4 - sin(θ)/θ**5 - 4*cos(θ)/θ**6 + 4/θ**6


-1/θ**4 - sin(θ)/θ**5 - 4*cos(θ)/θ**6 + 4/θ**6

In [47]:
show((C_se3.diff()/th).expand())

1/(3*θ**4) - cos(θ)/θ**6 - 4/θ**6 + 5*sin(θ)/θ**7


1/(3*θ**4) - cos(θ)/θ**6 - 4/θ**6 + 5*sin(θ)/θ**7

In [48]:
PA = W * V + V * W - w.dot(v) * W;
PB = W * W * V + V * W * W + w.dot(v) * (3 * W - W * W)
PC = -3 * w.dot(v) * W * W
Q = V / 2 + A * PA + B * PB + C * PC
dQ = diff_mat(Q, [vx, vy, vz, wx, wy, wz])
show(dQ.applyfunc(sp.simplify))

Matrix([[w.x()*(B + 3*C)*(w.y()**2 + w.z()**2), w.y()*(-2*A + B*(w.y()**2 + w.z()**2) + 3*C*(w.y()**2 + w.z()**2)), w.z()*(-2*A + B*(w.y()**2 + w.z()**2) + 3*C*(w.y()**2 + w.z()**2)), v.x()*(B + 3*C)*(w.y()**2 + w.z()**2), -2*A*v.y() + B*v.y()*(w.y()**2 + w.z()**2) + 2*B*w.y()*(v.x()*w.x() + v.y()*w.y() + v.z()*w.z()) + 3*C*(v.y()*w.y()**2 + v.y()*w.z()**2 + 2*w.y()*(v.x()*w.x() + v.y()*w.y() + v.z()*w.z())), -2*A*v.z() + B*v.z()*(w.y()**2 + w.z()**2) + 2*B*w.z()*(v.x()*w.x() + v.y()*w.y() + v.z()*w.z()) + 3*C*(v.z()*w.y()**2 + v.z()*w.z()**2 + 2*w.z()*(v.x()*w.x() + v.y()*w.y() + v.z()*w.z())), -A*(w.x()*w.z() - w.y()) - B*w.x()*(w.x()*w.y() - 2*w.z()) - 3*C*w.x()**2*w.y(), A*(w.x() - w.y()*w.z()) - B*w.y()*(w.x()*w.y() - 2*w.z()) - 3*C*w.x()*w.y()**2, -A*w.z()**2 - B*w.x()**2 - B*w.x()*w.y()*w.z() - B*w.y()**2 + B*w.z()**2 - 3*C*w.x()*w.y()*w.z() + 1/2, -A*(v.x()*w.z() - v.y()) - B*(v.x()*w.z() + v.x()*(w.x()*w.y() - 3*w.z()) + 2*v.z()*w.x() + w.y()*(v.x()*w.x() + v.y()*w.y() + v.z()

Matrix([
[                                          w.x()*(B + 3*C)*(w.y()**2 + w.z()**2),                                     w.y()*(-2*A + B*(w.y()**2 + w.z()**2) + 3*C*(w.y()**2 + w.z()**2)),                                    w.z()*(-2*A + B*(w.y()**2 + w.z()**2) + 3*C*(w.y()**2 + w.z()**2)),                                                                                                                                                                                      v.x()*(B + 3*C)*(w.y()**2 + w.z()**2),                              -2*A*v.y() + B*v.y()*(w.y()**2 + w.z()**2) + 2*B*w.y()*(v.x()*w.x() + v.y()*w.y() + v.z()*w.z()) + 3*C*(v.y()*w.y()**2 + v.y()*w.z()**2 + 2*w.y()*(v.x()*w.x() + v.y()*w.y() + v.z()*w.z())),                              -2*A*v.z() + B*v.z()*(w.y()**2 + w.z()**2) + 2*B*w.z()*(v.x()*w.x() + v.y()*w.y() + v.z()*w.z()) + 3*C*(v.z()*w.y()**2 + v.z()*w.z()**2 + 2*w.z()*(v.x()*w.x() + v.y()*w.y() + v.z()*w.z())),                       -A*(w.x()*w.z() - w.y(